<a href="https://colab.research.google.com/github/mwzkhalil/translation-finetuning-Seamless-/blob/main/Translation_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers datasets evaluate sacrebleu sentencepiece

In [2]:
from datasets import load_dataset
books = load_dataset("Helsinki-NLP/opus-100", "en-ur")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/753913 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
books = books['train'].select(range(100)).train_test_split(test_size=0.2)

In [18]:
books

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 80
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 20
    })
})

In [4]:
from transformers import AutoTokenizer

In [7]:
checkpoint = "facebook/hf-seamless-m4t-medium"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, src_lang='eng', tgt_lang='urd')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
source_lang = "en"
target_lang = "ur"
prefix = "translate English to Urdu: "

In [10]:
max_input_len=128
max_target_len=128
def tokenizer_fn(batch):
    inputs = [x[source_lang] for x in batch['translation']]
    targets = [x[target_lang] for x in batch['translation']]

    tokenized_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True)
    tokenized_targets = tokenizer(text_target=targets, max_length=max_target_len, truncation=True)

    tokenized_inputs['labels'] = tokenized_targets['input_ids']
    return tokenized_inputs

In [11]:
tokenized_books = books.map(tokenizer_fn, batched=True, remove_columns=books["train"].column_names)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
#!pip install transformers --upgrade

In [12]:
from transformers import SeamlessM4Tv2ForTextToText, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = SeamlessM4Tv2ForTextToText.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


config.json:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

You are using a model of type seamless_m4t to instantiate a model of type seamless_m4t_v2. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/5.35k [00:00<?, ?B/s]

In [13]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [14]:
import evaluate
metric = evaluate.load("sacrebleu")

In [15]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
# !pip install accelerate -U

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="seamless-tune-en-ur",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    eval_accumulation_steps=1,
    gradient_checkpointing=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model('checkpoint/my_saved_model')

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
